In [1]:
import pandas as pd
import numpy as np

data = {'Movie1':[3     , np.nan, 1     , np.nan, np.nan, 2     ],
        'Movie2':[np.nan, 2     , 2     , 1     , 4     , np.nan],
        'Movie3':[3     , 2     , np.nan, 3     , np.nan, 1     ],
        'Movie4':[np.nan, 5     , np.nan, np.nan, 1     , 5     ],
        'Movie5':[5     ,np.nan , 5     , 2     , 1     , np.nan]
        

       }
 

df = pd.DataFrame(data, index =['User1',
                                'User2',
                                'User3',
                                'User4',
                                'User5',
                                'User6'])
df

,Movie1,Movie2,Movie3,Movie4,Movie5
User1,3.0,NaN,3.0,NaN,5.0
User2,NaN,2.0,2.0,5.0,NaN
User3,1.0,2.0,NaN,NaN,5.0
User4,NaN,1.0,3.0,NaN,2.0
User5,NaN,4.0,NaN,1.0,1.0
User6,2.0,NaN,1.0,5.0,NaN


In [4]:
from itertools import repeat

tuple_ = []
for index, row in df.iterrows():
    #tuple_.append(row)
    tuple_.append(tuple(row))
(tuple_)    

main_dist = []
for first_row, other_row in zip(repeat(tuple_[0]),tuple_[1:]):
    
    inner_dist = []
    for i, j in zip(first_row, other_row):        
        
        if (i >= 1 and i <= 10) and (j >= 1 and j <= 10): 
            inner_dist.append([i,j])
    main_dist.append(inner_dist)
        
main_dist

i_list, j_list = [], []
for ele in main_dist:
    
    i_list1, j_list1 = [], []
    for a, b in ele:
        
        i_list1.append(a)
        j_list1.append(b)
        
    i_list.append(i_list1)
    j_list.append(j_list1)
print(len(i_list[0]), len(i_list[1]), len(i_list[2]))
print(len(j_list[0]), len(j_list[1]), len(j_list[2]))

1 2 2
1 2 2


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
# vec1 = np.array([[1,1,0,1,1]])
# vec2 = np.array([[0,1,0,1,1]])
#print(cosine_similarity([vec1, vec2]))
#print(cosine_similarity(vec1, vec2))

from collections import defaultdict

similarity_dict = defaultdict(dict)

similarity_list = []
for i, j in zip(i_list, j_list):
    similarity_list.append(cosine_similarity([i], [j]))
print(similarity_list)

for count, (sim, (k, v)) in enumerate(zip(similarity_list, df.items())):

    similarity_dict['Similarity_with_UserID_1'][f'User{count+2}'] = sim[0][0] # count + 2 because count starts from zero # sim[0][0] because float of similarity is packed within two ndarray layers
    
similarity_df = (pd.DataFrame(similarity_dict))

#Getting the UserID's of most similar to least similar Users
similarity_user_order = similarity_df.sort_values(by = 'Similarity_with_UserID_1', ascending = False).index
similarity_df_sorted = similarity_df.sort_values(by = 'Similarity_with_UserID_1', ascending = False)
similarity_df_sorted

[array([[1.]]), array([[0.94174191]]), array([[0.90373784]]), array([[1.]]), array([[0.9486833]])]


,Similarity_with_UserID_1
User2,1.000000
User5,1.000000
User6,0.948683
User3,0.941742
User4,0.903738


In [10]:
a = df
a = a.reset_index()

a_cols = a.columns[1:]
a.columns = ['User', 'Movie1', 'Movie2', 'Movie3', 'Movie4', 'Movie5']
a = a.set_index(['User'])
a

,Movie1,Movie2,Movie3,Movie4,Movie5
User,,,,,
User1,3.0,NaN,3.0,NaN,5.0
User2,NaN,2.0,2.0,5.0,NaN
User3,1.0,2.0,NaN,NaN,5.0
User4,NaN,1.0,3.0,NaN,2.0
User5,NaN,4.0,NaN,1.0,1.0
User6,2.0,NaN,1.0,5.0,NaN


In [12]:
unrated_movies = []
for index, row in a.iterrows():
    if index == 'User1':
        for i in a_cols:
            
            if pd.isnull(row[i]):
                unrated_movies.append(i)
    else:
        break
unrated_movies

['Movie2', 'Movie4']

In [14]:
list_of_movies_rated_by_others_and_not_user1 = []
for i in range(len(unrated_movies)):
    list_of_movies_rated_by_others_and_not_user1.append(a[unrated_movies[i]].dropna())
(list_of_movies_rated_by_others_and_not_user1) # .name to get name of series

[User
 User2    2.0
 User3    2.0
 User4    1.0
 User5    4.0
 Name: Movie2, dtype: float64,
 User
 User2    5.0
 User5    1.0
 User6    5.0
 Name: Movie4, dtype: float64]

In [15]:
for index, row in a.iterrows():
    if index == 'User1':
        for i in unrated_movies:
            
            for j in range(len(list_of_movies_rated_by_others_and_not_user1)):
                numerator, deno = 0, 0
                for index1, row1 in list_of_movies_rated_by_others_and_not_user1[j].iteritems():
                    
                    for index2, row2 in similarity_df.iterrows():
                        
                        if index2 == index1:
                            numerator += row2.Similarity_with_UserID_1 * row1
                            deno += row2.Similarity_with_UserID_1
                        else:
                            pass
            
                row[i] = numerator/deno 
    else:
        break
a

,Movie1,Movie2,Movie3,Movie4,Movie5
User,,,,,
User1,3.0,3.643462,3.0,3.643462,5.0
User2,NaN,2.000000,2.0,5.000000,NaN
User3,1.0,2.000000,NaN,NaN,5.0
User4,NaN,1.000000,3.0,NaN,2.0
User5,NaN,4.000000,NaN,1.000000,1.0
User6,2.0,NaN,1.0,5.000000,NaN


Thus we have generated the possible rating that User1 may give to the movies User1 has not watched yet. Based on a certain cutoff threshold between 1 and 5, we may recommend these unwatched movies to User1.